In [1]:
# coding: utf-8

import unittest
import requests
import re
from lxml import etree
from bs4 import BeautifulSoup
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

In [16]:
# 1) Ecrire une fonction get_prices_from_url() qui extrait des informations à partir des 2 pages ci-dessus.
# Exemple get_prices_from_url(URL_PAGE2) doit retourner :
# {'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
#  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
#  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}


def get_prices_from_url(url):
    
    r2 = requests.get(url)
    content2 = r2.content.decode('utf-8')
    
    rx_price = re.compile('<span class="pricing-table-price">\n([^<]+)<span>')
    rx_storage = re.compile('<li>([^<]+)file storage</li>\n')
    rx_database1 = re.compile('<li>([^<]+)database</li>')
    rx_database2 = re.compile('<li>([^<]+)databases</li>')
    
# Partie type du contrat 

    soup1 = BeautifulSoup(content2)
    titres = soup1.findAll('h2')
    list_type = []
    for titre in titres:
        list_type.append(titre.text)

    
# Partie Price

    match2_price = rx_price.search(content2)
    y = set()
    for match2_price in rx_price.finditer(content2):    
        y.add(match2_price.group(1))  
    price = list(y)
    price[0] = str(str.split(price[0])).replace("[","").replace("]","").replace("'","")
    price[1] = str(str.split(price[1])).replace("[","").replace("]","").replace("'","")
    price[2] = str(str.split(price[2])).replace("[","").replace("]","").replace("'","")
    if len(list_type) == 4:
        price[3] = str(str.split(price[3])).replace("[","").replace("]","").replace("'","")

# Partie Storage

    match2_storage = rx_storage.search(content2)

    y = set()
    for match2_storage in rx_storage.finditer(content2):    
        y.add(match2_storage.group(1))  
    storage = list(y)
    storage[0] = storage[0].replace(' ','')
    storage[1] = storage[1].replace(' ','')
    storage[2] = storage[2].replace(' ','')
    if len(list_type) == 4:
        storage[3] = storage[3].replace(' ','')
    

# Partie Database

    match2_database1 = rx_database1.search(content2)
    match2_database2 = rx_database2.search(content2)

    y = set()
    y.add(match2_database1.group(1))
    for match2_database2 in rx_database2.finditer(content2):    
        y.add(match2_database2.group(1))   
    database = list(y)
    database[0] = database[0].replace(' ','')
    database[1] = database[1].replace(' ','')
    database[2] = database[2].replace(' ','')
    if len(list_type) == 4:
        database[3] = database[3].replace(' ','')
    
    if len(list_type) == 3:
        prices = db =  {list_type[0]: {'price': price[1], 'storage': storage[0], 'databases': int(database[0])},
                    list_type[1]: {'price': price[0], 'storage': storage[2], 'databases': int(database[2])},
                    list_type[2]: {'price': price[2], 'storage': storage[1], 'databases': int(database[1])}}
        
    if len(list_type) == 4:
        prices = db =  {list_type[0]: {'price': price[2], 'storage': storage[0], 'databases': int(database[0])},
                    list_type[1]: {'price': price[0], 'storage': storage[2], 'databases': int(database[3])},
                    list_type[2]: {'price': price[3], 'storage': storage[1], 'databases': int(database[1])},
                    list_type[3]: {'price': price[1], 'storage': storage[3], 'databases': int(database[2])}}
        
    return prices

In [17]:
#get_prices_from_url(URL_PAGE2)
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

In [3]:
from bs4 import BeautifulSoup
# 2) Ecrire une fonction qui extrait des informations sur une bière de beowulf
# Exemple URL: https://www.beerwulf.com/fr-fr/p/bieres/brouwerij-t-verzet-super-noah.33

def extract_beer_infos(url):
    r4 = requests.get(url)
    content4 = r4.content.decode('utf-8')
    soup = BeautifulSoup(content4)
    
    name = soup.find('h1').text
    price2 = soup.find_all('span', {'class': "price"})[0].get_text()
    price2 = price2.split()[0]
    price2 = price2.replace(",", ".")
    rating = soup.find_all('div', {'class': "stars"})
    rating = str(rating).split(sep='"')
    rating = int(rating[3])
    v = soup.find_all('div', {'class': "product-subtext"})[0].get_text()
    words = v.split()
    volume = int(words[-2])
    
    infos = {
        'name': name,
        'note': rating,
        'price': price2,
        'volume': volume,
    }
    return infos


extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/brouwerij-t-verzet-super-noah.33')


{'name': "Brouwerij 'T Verzet Super NoAH",
 'note': 70,
 'price': '2.59',
 'volume': 33}

In [4]:
# Cette URL retourne un JSON avec une liste de bières
URL_BEERLIST_AUTRICHE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=Autriche&container=Bouteille"

# 3) Ecrire une fonction qui prend l'argument "url" retourne les informations sur une liste de bière via l'API de beowulf.
# Cette fonction doit retourner la liste des informations obtenues par la fonction extract_beer_infos() définie ci-dessus.
# Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.
#
# Exemple de retour :
# [{'name': 'Engelszell Benno', 'note': 70, 'price': 4.29, 'volume': 33}
#  {'name': 'Engelszell Trappisten Weiße', 'note': 70, 'price': 3.39, 'volume': 33}
#  {'name': 'Engelszell Gregorius', 'note': 70, 'price': 4.49, 'volume': 33}
#  {'name': 'Bevog Rudeen Black IPA', 'note': 80, 'price': 4.49, 'volume': 33}
#  {'name': 'Bevog Tak Pale Ale', 'note': 70, 'price': 2.79, 'volume': 33}
#  {'name': 'Brew Age Affenkönig', 'note': 70, 'price': 3.49, 'volume': 33}
#  {'name': 'Stiegl Goldbraü', 'note': 70, 'price': 2.49, 'volume': 33}
#  {'name': 'Stiegl Columbus 1492', 'note': 70, 'price': 2.49, 'volume': 33}
#  {'name': 'Brew Age Hopfenauflauf', 'note': 70, 'price': 2.99, 'volume': 33}]

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    r5 = requests.get(url)
    content5 = r5.content.decode('utf-8')
    soup5 = BeautifulSoup(content5)


    content_ref = soup5.find('p')
    content_ref = str(content_ref).split(sep=',')
    cont_ref = []

    for words in content_ref:
        if "contentReference" in words:
            cont_ref.append(words)

    beer_pages = []
    c = len('"contentReference":"')
    for a in cont_ref:
        b = a[c:]
        b = b.replace('"','')
        b = "https://www.beerwulf.com" + b
        beer_pages.append(b)


    # Sequential version (slow):
    beers = []
    i=0
    for x in beer_pages:
        beers.append(extract_beer_infos(beer_pages[i]))
        i = i +1

    # Parallel version (faster):
    # beers = []

    return beers

extract_beer_list_infos(URL_BEERLIST_AUTRICHE)

[{'name': 'Engelszell Trappisten Weiße',
  'note': 70,
  'price': '3.39',
  'volume': 33},
 {'name': 'Engelszell Benno', 'note': 70, 'price': '4.29', 'volume': 33},
 {'name': 'Engelszell Gregorius', 'note': 70, 'price': '4.49', 'volume': 33},
 {'name': 'Bevog Tak Pale Ale', 'note': 70, 'price': '2.79', 'volume': 33},
 {'name': 'Brew Age Affenkönig', 'note': 70, 'price': '3.49', 'volume': 33},
 {'name': 'Stiegl Goldbraü', 'note': 70, 'price': '2.49', 'volume': 33},
 {'name': 'Bevog Rudeen Black IPA', 'note': 80, 'price': '4.49', 'volume': 33},
 {'name': 'Stiegl Columbus 1492', 'note': 70, 'price': '2.49', 'volume': 33},
 {'name': 'Brew Age Hopfenauflauf', 'note': 70, 'price': '2.99', 'volume': 33}]

In [18]:
class Lesson3Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)

        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)

    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )

    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_AUTRICHE)
        # >Il y a 9 bières autrichiennes :
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 9)
        # toutes ont 33cl :
        for beer in infos:
            self.assertEqual(beer['volume'], 33)


def run_tests():
    test_suite = unittest.makeSuite(Lesson3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)


if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson3Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson3Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.952s

OK
